<a href="https://colab.research.google.com/github/BengIslam7/TinyML-SensorsValuesPrediction/blob/main/Anomaly_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip -q '/content/gdrive/MyDrive/iot_dataset.zip'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

In [ ]:
data = pd.read_csv('/content/DATA-large.CSV')

In [ ]:
data.head()

,time,temperature,humidity,pressure,lux
0,2022/3/12 22:32:2,24.2,32.7,98490.45,10.00
1,2022/3/12 22:32:6,24.0,33.9,98486.98,12.08
2,2022/3/12 22:32:11,24.1,33.9,98488.27,22.92
3,2022/3/12 22:32:16,24.1,33.8,98489.58,22.92
4,2022/3/12 22:32:21,24.1,35.0,98489.49,22.92


In [ ]:
temp = data['temperature'].values
real_temp = data['temperature'].values

In [ ]:
maxt=np.max(temp)
mint=np.min(temp)

In [ ]:
maxt

np.float64(30.0)

In [ ]:
mint

np.float64(19.0)

In [ ]:
temp

array([24.2, 24. , 24.1, ..., 23.3, 23.3, 23.3])

In [ ]:
temp.shape

(693220,)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
temp = scaler.fit_transform(temp.reshape(-1,1))

In [ ]:
temp

array([[1.9202781 ],
       [1.72800735],
       [1.82414273],
       ...,
       [1.05505973],
       [1.05505973],
       [1.05505973]])

In [ ]:
temp.shape

(693220, 1)

In [ ]:
X = []
for i in range(0,temp.shape[0]-10,10):
  X.append(temp[i:i+10])
X=np.array(X)
X = X.reshape(X.shape[0], -1)
print(X.shape)

(69321, 10)


In [ ]:
# Parameters
n_estimators = 200  # Number of trees
contamination = 0.01  # Expected proportion of anomalies
sample_size = 256  # Number of samples used to train each tree

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
model = IsolationForest(n_estimators=n_estimators,
                            contamination=contamination,
                            max_samples=sample_size,
                            random_state=42)
#model = OneClassSVM(kernel="rbf", degree=3, gamma=0.1, nu=0.01)
model.fit(X)


IsolationForest(contamination=0.01, max_samples=256, n_estimators=200,
                random_state=42)

In [ ]:
windows = []
for i in range(692720,693220-10):
  window = temp[i:i+10].reshape(1, -1)
  windows.append(window)

In [ ]:
windows = np.array(windows)

In [ ]:
windows.shape

(490, 1, 10)

In [ ]:
windows=windows.reshape(windows.shape[0],windows.shape[2])

In [ ]:
windows

array([[0.37272727, 0.37272727, 0.37272727, ..., 0.37272727, 0.37272727,
        0.37272727],
       [0.37272727, 0.37272727, 0.37272727, ..., 0.37272727, 0.37272727,
        0.37272727],
       [0.37272727, 0.37272727, 0.37272727, ..., 0.37272727, 0.37272727,
        0.37272727],
       ...,
       [0.39090909, 0.39090909, 0.39090909, ..., 0.39090909, 0.39090909,
        0.39090909],
       [0.39090909, 0.39090909, 0.39090909, ..., 0.39090909, 0.39090909,
        0.39090909],
       [0.39090909, 0.39090909, 0.39090909, ..., 0.39090909, 0.39090909,
        0.39090909]])

In [ ]:
scores = model.decision_function(windows)

In [ ]:
scores

array([0.21686438, 0.21556994, 0.22040049, 0.21836646, 0.21018188,
       0.2170566 , 0.21688788, 0.21686438, 0.21556994, 0.22040049,
       0.21836646, 0.21018188, 0.2170566 , 0.21688788, 0.21686438,
       0.21556994, 0.22040049, 0.21836646, 0.21929668, 0.22110561,
       0.22039604, 0.22519897, 0.22519897, 0.22519897, 0.22519897,
       0.22519897, 0.22519897, 0.22519897, 0.22519897, 0.22519897,
       0.22519897, 0.21030786, 0.21873951, 0.22046755, 0.21686438,
       0.21556994, 0.22040049, 0.21836646, 0.21929668, 0.22110561,
       0.22039604, 0.22519897, 0.22519897, 0.21030786, 0.20537837,
       0.21596259, 0.2127444 , 0.20406873, 0.2127317 , 0.21132541,
       0.20755995, 0.20678797, 0.20551335, 0.20138879, 0.20924711,
       0.20315705, 0.21234214, 0.19442696, 0.2054158 , 0.2056672 ,
       0.20383959, 0.20742599, 0.21226336, 0.21585994, 0.20835128,
       0.2170566 , 0.21688788, 0.21686438, 0.21556994, 0.20535222,
       0.20027747, 0.21168703, 0.2089238 , 0.20943626, 0.21294

In [ ]:
len(scores)

490

In [ ]:
anomaly = model.predict(windows)

In [ ]:
anomaly

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
test = [12.0 , -1.8 , -2.2 , 29.3 , -10.5, 31.8 , -5.7 , 33.8, 34.9 , -10.9]
#test = [21.1,21.3,21.3,21.4,21.5,21.7,21.7,21.8,21.9,22.0]

In [424]:
test = [
    21.2, 21.3, 21.4, 21.5, 21.6,
    21.7, 21.8, 21.9, 22.0, 22.1
]

In [425]:
test = np.array(test)

In [426]:
from sklearn.preprocessing import StandardScaler
test = scaler.transform(test.reshape(-1,1))

In [427]:
test.shape

(10, 1)

In [428]:
test = test.reshape(1,-1)

In [429]:
test

array([[-0.96378313, -0.86764776, -0.77151238, -0.67537701, -0.57924163,
        -0.48310626, -0.38697088, -0.29083551, -0.19470014, -0.09856476]])

In [430]:
test.shape

(1, 10)

In [431]:
scores = model.decision_function(test)

In [432]:
scores

array([0.16557757])

In [433]:
anomaly = model.predict(test)

In [435]:
if anomaly == 1:
  print("Normal")
else:
  print("Anomaly")

Normal
